In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix


import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, accuracy_score

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
df = pd.read_csv('dataP.csv')
df['urgency'] = df['urgency'].fillna(0)
texts = df["letter_text"].astype(str).values
labels_type = df["referal_type"].astype(str).values
labels_urgency = df["urgency"].astype(str).values
#Split the data
X = df['letter_text']
y = df[['referal_type', 'urgency']]

# Functional API

## Full Dataset

In [ ]:
#Label Encoding
label_encoder_type = LabelEncoder()
labels_type_encoded = label_encoder_type.fit_transform(labels_type)
num_classes_type = len(label_encoder_type.classes_)

label_encoder_urgency = LabelEncoder()
labels_urgency_encoded = label_encoder_urgency.fit_transform(labels_urgency)
num_classes_urgency = len(label_encoder_urgency.classes_)

In [ ]:
#Tokenisation and Sequencing
max_words = 10000
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
max_len = max(len(s) for s in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding="post")

In [ ]:
#train and test split
X_train, X_test, y_train_type, y_test_type, y_train_urgency, y_test_urgency = train_test_split(
    padded_sequences,
    labels_type_encoded,
    labels_urgency_encoded,
    test_size=0.2,
    random_state=1
)

In [ ]:
# Define the input layer
inputs = keras.Input(shape=(max_len,))

# Shared layers - the core of the model
embedding_layer = keras.layers.Embedding(max_words, 128)(inputs)
pooling_layer = keras.layers.GlobalAveragePooling1D()(embedding_layer)

# Add dropout for regularization
dropout_1 = keras.layers.Dropout(0.5)(pooling_layer)

# Dense layer with L2 regularization to prevent overfitting
shared_dense_layer = keras.layers.Dense(
    32, 
    activation="relu",
    kernel_regularizer=l2(0.001)
)(dropout_1)

# Additional dropout before output layers
dropout_2 = keras.layers.Dropout(0.3)(shared_dense_layer)

# Output layer for referral type prediction
type_output = keras.layers.Dense(
    num_classes_type, activation="softmax", name="type_output"
)(dropout_2)

# Output layer for urgency prediction
urgency_output = keras.layers.Dense(
    num_classes_urgency, activation="softmax", name="urgency_output"
)(dropout_2)

# Build the complete model
model = keras.Model(inputs=inputs, outputs=[type_output, urgency_output])

# Use Adam optimizer with lower learning rate
optimizer = keras.optimizers.Adam(learning_rate=0.0005)

# Compile with different loss weights for each task
model.compile(
    optimizer=optimizer,
    loss={
        "type_output": "sparse_categorical_crossentropy",
        "urgency_output": "sparse_categorical_crossentropy"
    },
    loss_weights={
        "type_output": 0.4,      # Less weight for type prediction
        "urgency_output": 1.6    # More weight for urgency (harder task)
    },
    metrics={
        "type_output": "accuracy",
        "urgency_output": "accuracy"
    }
)

# Set up early stopping to prevent overfitting
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Train the model
history = model.fit(
    X_train,
    y={"type_output": y_train_type, "urgency_output": y_train_urgency},
    epochs=100,
    batch_size=32,
    validation_data=(X_test, {"type_output": y_test_type, "urgency_output": y_test_urgency}),
    callbacks=[early_stopping]
)

In [ ]:
history_dict = history.history

#Plot for Referral Type
type_acc = history_dict['type_output_accuracy']
val_type_acc = history_dict['val_type_output_accuracy']
type_loss = history_dict['type_output_loss']
val_type_loss = history_dict['val_type_output_loss']
epochs = range(1, len(type_acc) + 1)

plt.figure(figsize=(12, 5))

# Plot Type Accuracy
plt.subplot(1, 2, 1)
plt.plot(epochs, type_acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_type_acc, 'b', label='Validation accuracy')
plt.title('Training and Validation Accuracy (Referral Type)')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plot Type Loss
plt.subplot(1, 2, 2)
plt.plot(epochs, type_loss, 'ro', label='Training loss')
plt.plot(epochs, val_type_loss, 'r', label='Validation loss')
plt.title('Training and Validation Loss (Referral Type)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.suptitle('Performance for Referral Type Output', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()


# Plot for Urgency
urgency_acc = history_dict['urgency_output_accuracy']
val_urgency_acc = history_dict['val_urgency_output_accuracy']
urgency_loss = history_dict['urgency_output_loss']
val_urgency_loss = history_dict['val_urgency_output_loss']

plt.figure(figsize=(12, 5))

# Plot Urgency Accuracy
plt.subplot(1, 2, 1)
plt.plot(epochs, urgency_acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_urgency_acc, 'b', label='Validation accuracy')
plt.title('Training and Validation Accuracy (Urgency)')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plot Urgency Loss
plt.subplot(1, 2, 2)
plt.plot(epochs, urgency_loss, 'ro', label='Training loss')
plt.plot(epochs, val_urgency_loss, 'r', label='Validation loss')
plt.title('Training and Validation Loss (Urgency)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.suptitle('Performance for Urgency Output', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

# predictions for the test set.
predictions = model.predict(X_test)

# Find highest probability for each output.
predicted_labels_type = np.argmax(predictions[0], axis=1)
predicted_labels_urgency = np.argmax(predictions[1], axis=1)

# Classifciation report for each output.

print("Final Performance Report on Test Data")

print("\nClassification Report for 'Referral Type'")
print(classification_report(
    y_test_type,
    predicted_labels_type,
    target_names=label_encoder_type.classes_,
    zero_division=0
))

print("\nClassification Report for 'Urgency'")
print(classification_report(
    y_test_urgency,
    predicted_labels_urgency,
    target_names=label_encoder_urgency.classes_,
    zero_division=0
))

# Confusion Matrix for 'Type'
cm_type = confusion_matrix(y_test_type, predicted_labels_type)
plt.figure(figsize=(6, 5))
sns.heatmap(cm_type, annot=True, fmt='d', cmap='Blues',
            xticklabels=label_encoder_type.classes_,
            yticklabels=label_encoder_type.classes_)
plt.title("Confusion Matrix: Referral Type")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

# Confusion Matrix for 'Urgency'
cm_urgency = confusion_matrix(y_test_urgency, predicted_labels_urgency)
plt.figure(figsize=(6, 5))
sns.heatmap(cm_urgency, annot=True, fmt='d', cmap='Oranges',
            xticklabels=label_encoder_urgency.classes_,
            yticklabels=label_encoder_urgency.classes_)
plt.title("Confusion Matrix: Urgency")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

## 5-Fold Cross-Validation

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Lists to collect scores
f1_scores_type = []
f1_scores_urgency = []
acc_scores_type = []
acc_scores_urgency = []

fold = 1
for train_index, val_index in kf.split(X):
    print(f"\n=== Fold {fold} ===")
    fold += 1

    # Split data
    X_train_fold = X.iloc[train_index]
    X_val_fold = X.iloc[val_index]
    y_train_type_fold = y['referal_type'].iloc[train_index]
    y_train_urgency_fold = y['urgency'].iloc[train_index]
    y_val_type_fold = y['referal_type'].iloc[val_index]
    y_val_urgency_fold = y['urgency'].iloc[val_index]

    # Create new label encoders for each fold
    label_encoder_type_fold = LabelEncoder()
    label_encoder_urgency_fold = LabelEncoder()

    # Fit on training data only
    label_encoder_type_fold.fit(y_train_type_fold)
    label_encoder_urgency_fold.fit(y_train_urgency_fold)

    # Update num_classes for this fold
    num_classes_type_fold = len(label_encoder_type_fold.classes_)
    num_classes_urgency_fold = len(label_encoder_urgency_fold.classes_)

    # Tokenize and pad
    X_train_seq = tokenizer.texts_to_sequences(X_train_fold)
    X_val_seq = tokenizer.texts_to_sequences(X_val_fold)
    X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
    X_val_pad = pad_sequences(X_val_seq, maxlen=max_len)

    # Build model
    inputs = keras.Input(shape=(max_len,))
    embedding_layer = keras.layers.Embedding(max_words, 128)(inputs)
    pooling_layer = keras.layers.GlobalAveragePooling1D()(embedding_layer)
    dropout_1 = keras.layers.Dropout(0.5)(pooling_layer)
    shared_dense_layer = keras.layers.Dense(
        32, activation="relu", kernel_regularizer=l2(0.001)
    )(dropout_1)
    dropout_2 = keras.layers.Dropout(0.3)(shared_dense_layer)

    type_output = keras.layers.Dense(num_classes_type_fold, activation="softmax", name="type_output")(dropout_2)
    urgency_output = keras.layers.Dense(num_classes_urgency_fold, activation="softmax", name="urgency_output")(dropout_2)

    model = keras.Model(inputs=inputs, outputs=[type_output, urgency_output])

    # Compile model
    optimizer = keras.optimizers.Adam(learning_rate=0.0005)
    model.compile(
        optimizer=optimizer,
        loss={
            "type_output": "sparse_categorical_crossentropy",
            "urgency_output": "sparse_categorical_crossentropy"
        },
        loss_weights={
            "type_output": 0.4,
            "urgency_output": 1.6
        },
        metrics={
            "type_output": "accuracy",
            "urgency_output": "accuracy"
        }
    )

    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Encode string labels into integers using fold-specific encoders
    y_train_type_fold = label_encoder_type_fold.transform(y_train_type_fold)
    y_train_urgency_fold = label_encoder_urgency_fold.transform(y_train_urgency_fold)
    y_val_type_fold = label_encoder_type_fold.transform(y_val_type_fold)
    y_val_urgency_fold = label_encoder_urgency_fold.transform(y_val_urgency_fold)

    y_train_type_fold = np.array(y_train_type_fold).astype('int32')
    y_train_urgency_fold = np.array(y_train_urgency_fold).astype('int32')
    y_val_type_fold = np.array(y_val_type_fold).astype('int32')
    y_val_urgency_fold = np.array(y_val_urgency_fold).astype('int32')

    # Train
    model.fit(
        X_train_pad,
        {"type_output": y_train_type_fold, "urgency_output": y_train_urgency_fold},
        validation_data=(X_val_pad, {"type_output": y_val_type_fold, "urgency_output": y_val_urgency_fold}),
        epochs=100,
        batch_size=32,
        verbose=0,
        callbacks=[early_stopping]
    )

    # Predict
    predictions = model.predict(X_val_pad)
    pred_type = np.argmax(predictions[0], axis=1)
    pred_urgency = np.argmax(predictions[1], axis=1)

    # Evaluate
    f1_type = f1_score(y_val_type_fold, pred_type, average='macro')
    f1_urgency = f1_score(y_val_urgency_fold, pred_urgency, average='macro')
    acc_type = accuracy_score(y_val_type_fold, pred_type)
    acc_urgency = accuracy_score(y_val_urgency_fold, pred_urgency)

   
    f1_scores_type.append(f1_type)
    f1_scores_urgency.append(f1_urgency)
    acc_scores_type.append(acc_type)
    acc_scores_urgency.append(acc_urgency)

    print(f"Referral Type → F1: {f1_type:.4f}, Accuracy: {acc_type:.4f}")
    print(f"Urgency       → F1: {f1_urgency:.4f}, Accuracy: {acc_urgency:.4f}")


print("\nK-Fold Final Results")
print("Referral Type F1 scores:", f1_scores_type)
print("Referral Type Acc scores:", acc_scores_type)
print(f"Referral Type - Mean F1: {np.mean(f1_scores_type):.4f}, Mean Acc: {np.mean(acc_scores_type):.4f}")

print("\nUrgency F1 scores:", f1_scores_urgency)
print("Urgency Acc scores:", acc_scores_urgency)
print(f"Urgency - Mean F1: {np.mean(f1_scores_urgency):.4f}, Mean Acc: {np.mean(acc_scores_urgency):.4f}")

## Vascular-Only Dataset

In [ ]:
df = pd.read_csv('dataP.csv')
df = df[df['referal_type'] != 'non vascular']
df['referal_type'].value_counts()


In [ ]:
texts = df["letter_text"].astype(str).values
labels_type = df["referal_type"].astype(str).values
labels_urgency = df["urgency"].astype(str).values

X = df['letter_text']
y = df[['referal_type', 'urgency']]

In [ ]:
#Label Encoding
label_encoder_type = LabelEncoder()
labels_type_encoded = label_encoder_type.fit_transform(labels_type)
num_classes_type = len(label_encoder_type.classes_)

label_encoder_urgency = LabelEncoder()
labels_urgency_encoded = label_encoder_urgency.fit_transform(labels_urgency)
num_classes_urgency = len(label_encoder_urgency.classes_)

In [ ]:
#Tokenisation and Sequencing
max_words = 10000
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
max_len = max(len(s) for s in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding="post")

In [ ]:
#train and test split
X_train, X_test, y_train_type, y_test_type, y_train_urgency, y_test_urgency = train_test_split(
    padded_sequences,
    labels_type_encoded,
    labels_urgency_encoded,
    test_size=0.2,
    random_state=1
)

In [ ]:
# Define the input layer
inputs = keras.Input(shape=(max_len,))

# Shared layers - the core of the model
embedding_layer = keras.layers.Embedding(max_words, 128)(inputs)
pooling_layer = keras.layers.GlobalAveragePooling1D()(embedding_layer)

# Add dropout for regularization
dropout_1 = keras.layers.Dropout(0.5)(pooling_layer)

# Dense layer with L2 regularization to prevent overfitting
shared_dense_layer = keras.layers.Dense(
    32,  
    activation="relu",
    kernel_regularizer=l2(0.001)
)(dropout_1)

# Additional dropout before output layers
dropout_2 = keras.layers.Dropout(0.3)(shared_dense_layer)

# Output layer for referral type prediction
type_output = keras.layers.Dense(
    num_classes_type, activation="softmax", name="type_output"
)(dropout_2)

# Output layer for urgency prediction
urgency_output = keras.layers.Dense(
    num_classes_urgency, activation="softmax", name="urgency_output"
)(dropout_2)

# Build the complete model
model = keras.Model(inputs=inputs, outputs=[type_output, urgency_output])

# Use Adam optimizer with lower learning rate
optimizer = keras.optimizers.Adam(learning_rate=0.0005)

# Compile with different loss weights for each task
model.compile(
    optimizer=optimizer,
    loss={
        "type_output": "sparse_categorical_crossentropy",
        "urgency_output": "sparse_categorical_crossentropy"
    },
    loss_weights={
        "type_output": 0.4,      # Less weight for type prediction
        "urgency_output": 1.6    # More weight for urgency (harder task)
    },
    metrics={
        "type_output": "accuracy",
        "urgency_output": "accuracy"
    }
)

# Set up early stopping to prevent overfitting
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Train the model
history = model.fit(
    X_train,
    y={"type_output": y_train_type, "urgency_output": y_train_urgency},
    epochs=100,
    batch_size=32,
    validation_data=(X_test, {"type_output": y_test_type, "urgency_output": y_test_urgency}),
    callbacks=[early_stopping]
)

In [ ]:
history_dict = history.history

#Plot for Referral Type
type_acc = history_dict['type_output_accuracy']
val_type_acc = history_dict['val_type_output_accuracy']
type_loss = history_dict['type_output_loss']
val_type_loss = history_dict['val_type_output_loss']
epochs = range(1, len(type_acc) + 1)

plt.figure(figsize=(12, 5))

# Plot Type Accuracy
plt.subplot(1, 2, 1)
plt.plot(epochs, type_acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_type_acc, 'b', label='Validation accuracy')
plt.title('Training and Validation Accuracy (Referral Type)')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plot Type Loss
plt.subplot(1, 2, 2)
plt.plot(epochs, type_loss, 'ro', label='Training loss')
plt.plot(epochs, val_type_loss, 'r', label='Validation loss')
plt.title('Training and Validation Loss (Referral Type)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.suptitle('Performance for Referral Type Output', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()


# Plot for Urgency
urgency_acc = history_dict['urgency_output_accuracy']
val_urgency_acc = history_dict['val_urgency_output_accuracy']
urgency_loss = history_dict['urgency_output_loss']
val_urgency_loss = history_dict['val_urgency_output_loss']

plt.figure(figsize=(12, 5))

# Plot Urgency Accuracy
plt.subplot(1, 2, 1)
plt.plot(epochs, urgency_acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_urgency_acc, 'b', label='Validation accuracy')
plt.title('Training and Validation Accuracy (Urgency)')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plot Urgency Loss
plt.subplot(1, 2, 2)
plt.plot(epochs, urgency_loss, 'ro', label='Training loss')
plt.plot(epochs, val_urgency_loss, 'r', label='Validation loss')
plt.title('Training and Validation Loss (Urgency)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.suptitle('Performance for Urgency Output', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

# predictions for the test set.
predictions = model.predict(X_test)

# Find highest probability for each output.
predicted_labels_type = np.argmax(predictions[0], axis=1)
predicted_labels_urgency = np.argmax(predictions[1], axis=1)

# Classifciation report for each output.

print("Final Performance Report on Test Data")

print("\nClassification Report for 'Referral Type'")
print(classification_report(
    y_test_type,
    predicted_labels_type,
    target_names=label_encoder_type.classes_,
    zero_division=0
))

print("\nClassification Report for 'Urgency'")
print(classification_report(
    y_test_urgency,
    predicted_labels_urgency,
    target_names=label_encoder_urgency.classes_,
    zero_division=0
))

# Confusion Matrix for 'Type'
cm_type = confusion_matrix(y_test_type, predicted_labels_type)
plt.figure(figsize=(6, 5))
sns.heatmap(cm_type, annot=True, fmt='d', cmap='Blues',
            xticklabels=label_encoder_type.classes_,
            yticklabels=label_encoder_type.classes_)
plt.title("Confusion Matrix: Referral Type")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

# Confusion Matrix for 'Urgency'
cm_urgency = confusion_matrix(y_test_urgency, predicted_labels_urgency)
plt.figure(figsize=(6, 5))
sns.heatmap(cm_urgency, annot=True, fmt='d', cmap='Oranges',
            xticklabels=label_encoder_urgency.classes_,
            yticklabels=label_encoder_urgency.classes_)
plt.title("Confusion Matrix: Urgency")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

## 5-Fold Cross-Validation

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Lists to collect scores
f1_scores_type = []
f1_scores_urgency = []
acc_scores_type = []
acc_scores_urgency = []

fold = 1
for train_index, val_index in kf.split(X):
    print(f"\n=== Fold {fold} ===")
    fold += 1

    # Split data
    X_train_fold = X.iloc[train_index]
    X_val_fold = X.iloc[val_index]
    y_train_type_fold = y['referal_type'].iloc[train_index]
    y_train_urgency_fold = y['urgency'].iloc[train_index]
    y_val_type_fold = y['referal_type'].iloc[val_index]
    y_val_urgency_fold = y['urgency'].iloc[val_index]

    # Create new label encoders for each fold
    label_encoder_type_fold = LabelEncoder()
    label_encoder_urgency_fold = LabelEncoder()

    # Fit on training data only
    label_encoder_type_fold.fit(y_train_type_fold)
    label_encoder_urgency_fold.fit(y_train_urgency_fold)

    # Update num_classes for this fold
    num_classes_type_fold = len(label_encoder_type_fold.classes_)
    num_classes_urgency_fold = len(label_encoder_urgency_fold.classes_)

    # Tokenize and pad
    X_train_seq = tokenizer.texts_to_sequences(X_train_fold)
    X_val_seq = tokenizer.texts_to_sequences(X_val_fold)
    X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
    X_val_pad = pad_sequences(X_val_seq, maxlen=max_len)

    # Build model
    inputs = keras.Input(shape=(max_len,))
    embedding_layer = keras.layers.Embedding(max_words, 128)(inputs)
    pooling_layer = keras.layers.GlobalAveragePooling1D()(embedding_layer)
    dropout_1 = keras.layers.Dropout(0.5)(pooling_layer)
    shared_dense_layer = keras.layers.Dense(
        32, activation="relu", kernel_regularizer=l2(0.001)
    )(dropout_1)
    dropout_2 = keras.layers.Dropout(0.3)(shared_dense_layer)

    type_output = keras.layers.Dense(num_classes_type_fold, activation="softmax", name="type_output")(dropout_2)
    urgency_output = keras.layers.Dense(num_classes_urgency_fold, activation="softmax", name="urgency_output")(dropout_2)

    model = keras.Model(inputs=inputs, outputs=[type_output, urgency_output])

    # Compile model
    optimizer = keras.optimizers.Adam(learning_rate=0.0005)
    model.compile(
        optimizer=optimizer,
        loss={
            "type_output": "sparse_categorical_crossentropy",
            "urgency_output": "sparse_categorical_crossentropy"
        },
        loss_weights={
            "type_output": 0.4,
            "urgency_output": 1.6
        },
        metrics={
            "type_output": "accuracy",
            "urgency_output": "accuracy"
        }
    )

    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Encode string labels into integers using fold-specific encoders
    y_train_type_fold = label_encoder_type_fold.transform(y_train_type_fold)
    y_train_urgency_fold = label_encoder_urgency_fold.transform(y_train_urgency_fold)
    y_val_type_fold = label_encoder_type_fold.transform(y_val_type_fold)
    y_val_urgency_fold = label_encoder_urgency_fold.transform(y_val_urgency_fold)

    y_train_type_fold = np.array(y_train_type_fold).astype('int32')
    y_train_urgency_fold = np.array(y_train_urgency_fold).astype('int32')
    y_val_type_fold = np.array(y_val_type_fold).astype('int32')
    y_val_urgency_fold = np.array(y_val_urgency_fold).astype('int32')

    # Train
    model.fit(
        X_train_pad,
        {"type_output": y_train_type_fold, "urgency_output": y_train_urgency_fold},
        validation_data=(X_val_pad, {"type_output": y_val_type_fold, "urgency_output": y_val_urgency_fold}),
        epochs=100,
        batch_size=32,
        verbose=0,
        callbacks=[early_stopping]
    )

    # Predict
    predictions = model.predict(X_val_pad)
    pred_type = np.argmax(predictions[0], axis=1)
    pred_urgency = np.argmax(predictions[1], axis=1)

    # Evaluate
    f1_type = f1_score(y_val_type_fold, pred_type, average='macro')
    f1_urgency = f1_score(y_val_urgency_fold, pred_urgency, average='macro')
    acc_type = accuracy_score(y_val_type_fold, pred_type)
    acc_urgency = accuracy_score(y_val_urgency_fold, pred_urgency)

   
    f1_scores_type.append(f1_type)
    f1_scores_urgency.append(f1_urgency)
    acc_scores_type.append(acc_type)
    acc_scores_urgency.append(acc_urgency)

    print(f"Referral Type → F1: {f1_type:.4f}, Accuracy: {acc_type:.4f}")
    print(f"Urgency       → F1: {f1_urgency:.4f}, Accuracy: {acc_urgency:.4f}")


print("\nK-Fold Final Results")
print("Referral Type F1 scores:", f1_scores_type)
print("Referral Type Acc scores:", acc_scores_type)
print(f"Referral Type - Mean F1: {np.mean(f1_scores_type):.4f}, Mean Acc: {np.mean(acc_scores_type):.4f}")

print("\nUrgency F1 scores:", f1_scores_urgency)
print("Urgency Acc scores:", acc_scores_urgency)
print(f"Urgency - Mean F1: {np.mean(f1_scores_urgency):.4f}, Mean Acc: {np.mean(acc_scores_urgency):.4f}")